In [10]:
# Импортируем функцию urlopen и request из модуля request пакета urllib
from urllib.request import urlopen
from urllib.request import Request
# Импортируем модуль errors
from urllib.error import HTTPError
# Импортируем "прекрасный суп"
from bs4 import BeautifulSoup
import re

In [11]:
# Напишем адрес, который будет запрашиваться при поиске. Адрес - результат выполнения функции
def search(KeyWords):
    search_page = "https://www.sciencedirect.com/search?qs="
    kw_position = 0
    while kw_position <= len(KeyWords)-1:
        search_page += KeyWords[kw_position] + "%20"
        kw_position += 1
    return search_page


In [12]:
# Поскольку сайт, который мы собираемся скрапить, защищен он ботов, необходимо доказать, что мы из мяса. Делаем это с помощью 
# оьъекта класса reqssion
import requests as req
session = req.Session()
# Пишем HTTP-заголовки. Отправляются при каждом запросе. ЗАМЕНИТЬ НА СВОИ ПРИ НЕОБХОДИМОСТИ
heads = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36",
         "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}


In [14]:
# Определяем функцию, которая будет запрашивать содержимое, находящееся по URL-адресу
# Функция учитывает возможные ошибки HTTPError или неправильное обращение к тегу (атрибуту тега) HTML

def get_anything(url):
    global session, heads
    try: # Посылаем запрос с помощью Сессионс, притворяясь человеком
        html = session.get(url, headers=heads)
    except HTTPError as e:
        return "Ошибка HTTP. Проверь URL адрес, либо у тебя нет права доступа"
    # Если запрос удачный, работаем с "супом"
    try:
        HTML_Code = BeautifulSoup(html.text)
        Result_Holders = HTML_Code.findAll("li", {"class":"ResultItem col-xs-24 push-m"}, limit = 5)
    except AttributeError as e:
        return "Такого ингредиента в супе нет!"
    for Result in Result_Holders:
        article_name = Result.h2
        authors = Result.ol.findAll("li")
        Journal = Result.find("div",{"class":"SubType hor"}).a.get_text()
        Link = Result.h2.a.get("href")
        print("List of authors:")
        for Author in authors:
            print(Author.get_text())
        print("Article name: ", article_name.get_text())
        print("Journal name: ", Journal)
        print("Article link: ","https://www.sciencedirect.com" + Link, 2*"\n")

# ВВОДИТЬ КЛЮЧЕВЫЕ СЛОВА ТУТ. ЗАТЕМ ЗАПУСКАТЬ ЭТО ОКНО.
MySearch = search(["gas", "condensate"])
get_anything(MySearch)

List of authors:
Maria Kirgina
Nataliya Belinskaya
Andrey Altynov
Aset Temirbolat
Article name:  Transformations of stable gas condensate hydrocarbons into high-octane gasoline components over ZSM-5 zeolite catalyst
Journal name:  Journal of Natural Gas Science and Engineering
Article link:  https://www.sciencedirect.com/science/article/pii/S1875510020304595 


List of authors:
Nader Faramarzi
Saeid Sadeghnejad
Article name:  Fluid and rock heterogeneity assessment of gas condensate reservoirs by wavelet transform of pressure-transient responses
Journal name:  Journal of Natural Gas Science and Engineering
Article link:  https://www.sciencedirect.com/science/article/pii/S1875510020303231 


List of authors:
Mutlaq ALAROUJ
Osamah ALOMAIR
Adel ELSHARKAWY
Article name:  Gas condensate reservoirs: Characterization and calculation of dew-point pressure
Journal name:  Petroleum Exploration and Development
Article link:  https://www.sciencedirect.com/science/article/pii/S1876380420601203 


L